## 4chan Scraper ##

This notebook will scrape a given board on 4chan at the moment, create a wordcloud of the most common words, and if desired, download all images associated with the 200 active threads and x number of replies on the board.

The idea behind this scraping is to feed the reply/subject text data into a machine learning model to analyze posts. The project came out of the desire to analyze the /biz/ board to determine which cryptocurrencies were becoming popular/which cryptocurrencies had a sustained buzz around them. This information could then be utilized in combination with a token sniper or other web3 application for a slight advantage in trading.

Image wise, this aspect is not fully fleshed out. The idea was to use neural networks to analyze and categorize types of memes on boards, but I have not had the time to train a model and am waiting for access to GPT-3's DALL-E rather than using what transformers are currently available. 

The image data is collected and stored in /imgs/, I'd like for the images to eventually be the input for both a trained classification neural network and a trained generative neural network.

In [1]:
import time
import pandas as pd
from urllib.request import urlopen
import json
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import requests
from PIL import Image
import re
import random
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import os
import openai
import spacy
from collections import Counter

openai.api_key = "OPEN-API-KEY"

def get_jsonparsed_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

# Collect general information about boards

url = (('https://a.4cdn.org/boards.json'))

data = get_jsonparsed_data(url)


board_names = []
board_titles = []
combined = []
pages = []

for i in range(0, 78):
    name = data['boards'][i]['board']
    title = data['boards'][i]['title']
    page = data['boards'][i]['pages']
    board_names.append(name)
    board_titles.append(title)
    pages.append(page)
    combination = name + " " + title
    combined.append(combination)

board_df = pd.DataFrame({'Name' : board_names, 'Title' : board_titles, 'Combined' : combined, 'Pages' : pages})

board_df

,Name,Title,Combined,Pages
0,3,3DCG,3 3DCG,10
1,a,Anime & Manga,a Anime & Manga,10
2,aco,Adult Cartoons,aco Adult Cartoons,10
3,adv,Advice,adv Advice,10
4,an,Animals & Nature,an Animals & Nature,10
...,...,...,...,...
73,wsg,Worksafe GIF,wsg Worksafe GIF,10
74,wsr,Worksafe Requests,wsr Worksafe Requests,10
75,x,Paranormal,x Paranormal,10
76,xs,Extreme Sports,xs Extreme Sports,10


In [2]:
# prompt user for input
board = input("Enter a board: ")
print(board)

biz


In [3]:
# Pulling Board information


def get_jsonparsed_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)


def get_numbers(df, i):  
    postno = df['Post Number'][i]
    replies = df['Replies'][i]
    return(postno, replies)

dataframe = pd.DataFrame(columns=['Subject', 'Comment', 'Post Number', 'Replies', 'reply_list', 'filename', 'tim_list', 'ext_list', 'combined'])
for i in range(10):
    i=i+1
    url = (("https://a.4cdn.org/") + str(board) + "/" + str(i) + '.json')
    data = get_jsonparsed_data(url)

    list = len(data['threads'])
   
    for i in range(0, list): 
      try:
          comment = data['threads'][i]['posts'][0]['com']
          subject = data['threads'][i]['posts'][0]['sub']
          postno = data['threads'][i]['posts'][0]['no']
          replies = data['threads'][i]['posts'][0]['replies']
      except KeyError:
          subject = 'No subject'
          postno = data['threads'][i]['posts'][0]['no']
          replies = data['threads'][i]['posts'][0]['replies']
      except KeyError:
          comment = data['threads'][i]['posts'][0]['sub']
      dataframe = dataframe.append({'Subject':subject, 'Comment':comment, 'Post Number':postno, 'Replies':replies}, ignore_index=True)
    time.sleep(1)
    i=i+1

dataframe = dataframe.fillna(0)

Above code produces dataframe seen below

In [4]:
dataframe.head(5)

,Subject,Comment,Post Number,Replies,reply_list,filename,tim_list,ext_list,combined
0,NO BEGGING,"<span style=""font-weight:600;font-size:150%;li...",4884770,0,0,0,0,0,0
1,Welcome to /biz/ - Business &amp; Finance,This board is for the discussion of topics rel...,21374000,1,0,0,0,0,0
2,No subject,F,45083736,1000,0,0,0,0,0
3,/AVAX/ - Avalanche General #278,"First of the Year Edition<br><br><span class=""...",45090694,288,0,0,0,0,0
4,/smg/ stock market general,"goodnight sweet prince<br><br><span class=""quo...",45105547,86,0,0,0,0,0


In [5]:
import sqlite3
con = sqlite3.connect('pdx911.sqlite3')
c = con.cursor()


In [6]:
dataframe.to_sql('board1', con=con, if_exists='append', index=False)

C:\Users\Bernie\anaconda3\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


### Grab reply text, image md5 hash (name), and img extension #

This function goes in and grabs all text and image data from replies in a given thread. It then drops the replies into a list in "reply_list", and drops the image name (it's md5 hash) and extension in the repsective columns
called "tim_list" and "ext_list". The "combined" column is the result of pairing the img name (md5) with the correlating extension.


In [7]:
for i in range(0,len(dataframe)):

  try:
    postno, replies = get_numbers(dataframe, i) 
    url = (("https://a.4cdn.org/" + str(board) + "/thread/") + str(postno) + '.json')
    data = get_jsonparsed_data(url)
    replies_text = []
    extensions = []
    images = []
    combined = []
    filenames = []
    for j in range(0,replies):
      try:
        reply = data['posts'][j]['com']
        img = data['posts'][j]['tim']
        ext = data['posts'][j]['ext']
        #file = data['posts'][i]['filename']
        #filenames.append(file)
        replies_text.append(reply)
        images.append(str(img))
        extensions.append(str(ext))
        combined.append(str(img)+str(ext))
        dataframe['reply_list'][i] = replies_text
        dataframe['tim_list'][i] = images
        dataframe['ext_list'][i] = extensions
        dataframe['combined'][i] = combined
        #dataframe['filename'][i] = filenames
      except KeyError:
        pass
      except IndexError: 
        pass
  except HTTPError:
    dataframe['reply_list'][i] = '404'
    dataframe['tim_list'][i] = '404'
    dataframe['ext_list'][i] = '404'
  else:
    pass
time.sleep(1)


C:\Users\Bernie\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Bernie\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Bernie\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Bernie\anaconda3\lib\site-package

The above will format the dataframe into the format below:

In [8]:
dataframe2 = dataframe.to_json(orient = 'split')

In [9]:
dataframe2[6][0]

'm'

In [10]:
import sqlite3
con = sqlite3.connect('4chan.sqlite3')
c = con.cursor()
dataframe.to_sql('boards', con=con, if_exists='append', index=False)

InterfaceError: Error binding parameter 4 - probably unsupported type.

In [ ]:
# # DOWNLOAD ALL IMAGES FOR THREAD/IN THREADS

# links = []
# newlinks = []
# dataframe['combined'] = dataframe['combined'].fillna(0)
# combined = dataframe['combined']

# for i in range(0, len(combined)):
#     if combined[i] == 0:
#         links.append('No image')
#     elif combined[i] != 0:
#         links = combined[i]
#         for i in links:
#             newlinks.append(i)
#             imgURL = ("https://i.4cdn.org/biz/") + str(i)
#             name = str(i)
#             print(imgURL)
#             r = requests.get(imgURL)
#             with open('imgs/'+name, 'wb') as f:
#                 f.write(r.content)
#             time.sleep(0)
# print(name)

In [ ]:
# directory = "imgs/"
# files_in_directory = os.listdir(directory)
# filtered_files = [file for file in files_in_directory if file.endswith(".webm")]

# for file in filtered_files:
# 	path_to_file = os.path.join(directory, file)
# 	os.remove(path_to_file)

In [ ]:
# path = "imgs/"
# dirs = os.listdir( path )
# final_size = 440;

# def resize_aspect_fit():
#     for item in dirs:
#          if item == '.DS_Store':
#              continue
#          if os.path.isfile(path+item):
#              im = Image.open(path+item)
#              f, e = os.path.splitext(path+item)
#              size = im.size
#              ratio = float(final_size) / max(size)
#              new_image_size = tuple([int(x*ratio) for x in size])
#              im = im.resize(new_image_size, Image.ANTIALIAS)
#              new_im = Image.new("RGB", (final_size, final_size))
#              new_im.paste(im, ((final_size-new_image_size[0])//2, (final_size-new_image_size[1])//2))
#              new_im.save(f + '_resized.jpg', 'JPEG', quality=90)
# resize_aspect_fit()

In [ ]:
# directory = "imgs/"
# files_in_directory = os.listdir(directory)
# filtered_files = [file for file in files_in_directory if not file.endswith("_resized.jpg")]

# for file in filtered_files:
# 	path_to_file = os.path.join(directory, file)
# 	os.remove(path_to_file)

In [ ]:
### THIS WILL CLEAN ALL DATA IN REPLY_TEXT ###

exploded_df = dataframe['reply_list'].explode().reset_index()

all_reply_test = exploded_df['reply_list']


all_replies = []
for i in range(0, len(all_reply_test)):
    result = re.sub("<(.*)>.*?|<(.*) />", " ", str(all_reply_test[i]))
    result = re.sub('&#039;', "'", result)
    result = re.sub('&quot;', '"', result)
    all_replies.append(result)

In [ ]:
## Append all subject and comment text to lists ##


subjects = []
comments = []
for i in range(0,len(dataframe)):
    subject_result =  re.sub("<(.*)>.*?|<(.*) />", " ", str(dataframe['Subject'][i]))
    subject_result = re.sub('&#039;', "'", subject_result)
    subject_result = re.sub('&quot;', '"', subject_result)
    subjects.append(subject_result)
    comment_result =  re.sub("<(.*)>.*?|<(.*) />", " ", str(dataframe['Comment'][i]))
    comment_result = re.sub('&#039;', "'", comment_result)
    comment_result = re.sub('&quot;', '"', comment_result)
    comments.append(comment_result)


In [ ]:
## ALL TEXT ON THE BOARD ##

import numpy as np

all_text_on_board = all_replies + subjects + comments

textdf = pd.DataFrame({'All text': all_text_on_board})

textdf = textdf.replace('0', np.nan)
textdf = textdf.replace(' ', np.nan)
textdf = textdf.dropna(how='all', axis=0).reset_index(drop=True)

textdf

In [ ]:
alltext_list = textdf['All text']

literally_all_text =[]
for i in range(0,len(alltext_list)):
    text = alltext_list[i]
    literally_all_text.append(text)

corp_string = ' '.join(map(str, literally_all_text))

corp_string = re.sub("\d+", "", corp_string)
#corp_string = re.sub(r'[\t\n ]+', ' ', corp_string)
#corp_string = re.sub(r"\b([^ ]|\d+)\b","",corp_string)
corp_string = re.sub("\\s+"," ", corp_string)
corp_string = re.sub(" ", " ", corp_string)

corp_string.strip()
corp_string.lower()



In [ ]:
corp_string = corp_string.lower()
nlp = spacy.load("en_core_web_sm")
doc = nlp(corp_string)
STOPWORDS = ["coin", "shit", "time", "year", "day", "fuck", "life", "way", "guys", "%", "days", "bros", "pic", "bull", "dump", "bags", "man", "rugs", "run", "lot", "fees", "home", "guy", "idea", "k", "moon", "pump", "bag", "scam"]

# all tokens that arent stop words or punctuations
words = [token.text
         for token in doc
         if not token.is_stop and not token.is_punct and len(token) >= 3]

# noun tokens that arent stop words or punctuations
nouns = [token.text
         for token in doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "NOUN")]

adjectives = [token.text
         for token in doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "ADJ")]

verbs = [token.text
         for token in doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "VERB")]

propn = [token.text
         for token in doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "PROPN")]
             
adposition = [token.text
         for token in doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "ADP")]

adverb = [token.text
         for token in doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "ADV")]
pronoun = [token.text
         for token in doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "PRON")]

determiner = [token.text
         for token in doc
         if (not token.is_punct and
             token.pos_ == "DET")]

conjunction = [token.text
         for token in doc
         if (token.pos_ == "CONJ")]

potential_coins = [token.text
         for token in doc
         if (not token.is_stop 
         and token.pos_ == "NOUN"
         and len(token) <= 4)]


# ```ADJ: adjective, e.g. big, old, green, incomprehensible, first
# ADP: adposition, e.g. in, to, during
# ADV: adverb, e.g. very, tomorrow, down, where, there
# AUX: auxiliary, e.g. is, has (done), will (do), should (do)
# CONJ: conjunction, e.g. and, or, but
# CCONJ: coordinating conjunction, e.g. and, or, but
# DET: determiner, e.g. a, an, the
# INTJ: interjection, e.g. psst, ouch, bravo, hello
# NOUN: noun, e.g. girl, cat, tree, air, beauty
# NUM: numeral, e.g. 1, 2017, one, seventy-seven, IV, MMXIV
# PART: particle, e.g. ’s, not,
# PRON: pronoun, e.g I, you, he, she, myself, themselves, somebody
# PROPN: proper noun, e.g. Mary, John, London, NATO, HBO
# PUNCT: punctuation, e.g. ., (, ), ?
# SCONJ: subordinating conjunction, e.g. if, while, that
# SYM: symbol, e.g. $, %, §, ©, +, −, ×, ÷, =, :), 😝
# VERB: verb, e.g. run, runs, running, eat, ate, eating
# X: other, e.g. sfpksdpsxmsa
# SPACE: space, e.g.````

# most common tokens
word_freq = Counter(words)
common_words = word_freq.most_common(100)

# most common noun tokens
noun_freq = Counter(nouns)
common_nouns = noun_freq.most_common(100)

# most common adjective tokens
adj_freq = Counter(adjectives)
common_adjectives = adj_freq.most_common(100)

# most common verb tokens
verb_freq = Counter(verbs)
common_verbs = verb_freq.most_common(100)

# most common propositions
prop_freq = Counter(propn)
common_props = prop_freq.most_common(100)

# most common adpostions
adposition_freq = Counter(propn)
common_adpositions = adposition_freq.most_common(100)

# most common adverbs
adverb_freq = Counter(adverb)
common_adverbs = adverb_freq.most_common(100)

# most common determiners
determiner_freq = Counter(determiner)
common_determiner = determiner_freq.most_common(100)

# most common pronouns
pronoun_freq = Counter(pronoun)
common_pronouns = pronoun_freq.most_common(100)

# most common conjunctions
conjunction_freq = Counter(conjunction)
common_conjunctions = conjunction_freq.most_common(100)

potential_coins_freq = Counter(potential_coins)
common_potential_coins = potential_coins_freq.most_common(100)


In [ ]:
#potential_coins
from nltk.probability import FreqDist
fdist = FreqDist(potential_coins)
common_potential_coins


In [ ]:
# Sample sentences
x = 10
y = len(common_pronouns)
z = len(common_conjunctions)


for i in range(0,7):
    sentence = common_props[(random.randrange(0, x, 1))][0] + " " + common_verbs[(random.randrange(0, x, 1))][0] + " " + common_verbs[(random.randrange(0, x, 1))][0] + " " +  common_adjectives[(random.randrange(0, x, 1))][0]+ " " + common_nouns[(random.randrange(0, x, 1))][0]
    sentence2 = common_props[(random.randrange(0, x, 1))][0]+ " " + common_nouns[(random.randrange(0, x, 1))][0] + " " + common_verbs[(random.randrange(0, x, 1))][0] + " " +  common_adjectives[(random.randrange(0, x, 1))][0] + " " + common_nouns[(random.randrange(0, x, 1))][0]
    sentence3 = common_pronouns[(random.randrange(0, y, 1))][0] + " " + common_verbs[(random.randrange(0, x, 1))][0]+ " " + common_adjectives[(random.randrange(0, x, 1))][0] + " " + common_nouns[(random.randrange(0, x, 1))][0]+ " " + common_pronouns[(random.randrange(0, y, 1))][0]+ " " + common_verbs[(random.randrange(0, x, 1))][0]+ " " + common_pronouns[(random.randrange(0, y, 1))][0]

    print(sentence)
    print(sentence2)


In [ ]:
# Wordcloud creation
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%m%d-%Y%H%M%S")

# Get all common words, map to string for wordcloud
wordcloud_common = []
for i in range(0,100):
    word = common_words[i][0]
    wordcloud_common.append(word)

wordcloud_string = ' '.join(map(str, wordcloud_common))

#corp_string_wordcloud = re.sub("https", "", corp_string)

wordcloud = WordCloud(width = 800, height = 800,
                background_color = 'white',
                stopwords=STOPWORDS,
                min_font_size = 10).generate(wordcloud_string)
 
# plot the WordCloud image    
                  
plt.figure(figsize = (10, 10), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 10)
plt.savefig(board + "" + dt_string + "" + ".png", format="png")
plt.show()


In [ ]:
corp_string

In [ ]:
# 'BAD WORDS' list to try data on GPT-3
# https://github.com/web-mech/badwords

url = ('https://raw.githubusercontent.com/web-mech/badwords/master/lib/lang.json')
data = get_jsonparsed_data(url)

badwords = []
for i in range(0, len(data['words'])):
    word  = data['words'][i]
    badwords.append(word)


text = corp_string
for badword in badwords:
        text = text.replace(badword,""*len(badword))

text



In [ ]:
split_text = text.split()
random.shuffle(split_text)

split_text

newtext = ' '.join(map(str, split_text))

newtext[:200]

In [ ]:
newtext[:800]

In [ ]:
## Explaining this board to a second grader
## Uses OPENAI

response = openai.Completion.create(
  engine="davinci",
  prompt="My fourth grader asked me what this passage means:\n\"\"\"\n" + newtext[:800] + "\"\"\"\nI rephrased it for him, in plain language a fourth grader can understand:\n\"\"\"\n",
  temperature=0.6,
  max_tokens=280,
  top_p=1,
  frequency_penalty=0.2,
  presence_penalty=0,
  stop=["\"\"\""]
)

print(response['choices'][0]['text'])

In [ ]:
plt.savefig(board + "" + '.png')


# def create_wordcloud():
# ''' Wordcloud creation:

# Get all common words from the board, map to string for wordcloud, generate a wordcloud 
# '''

# wordcloud_common = []
# for i in range(0,100):
#     word = common_words[i][0]
#     wordcloud_common.append(word)

# wordcloud_string = ' '.join(map(str, wordcloud_common))

# #corp_string_wordcloud = re.sub("https", "", corp_string)

# wordcloud = WordCloud(width = 800, height = 800,
#                 background_color = 'white',
#                 stopwords=STOPWORDS,
#                 min_font_size = 10).generate(wordcloud_string)
 
# # plot the WordCloud image                      
# plt.figure(figsize = (8, 8), facecolor = None)
# plt.imshow(wordcloud)
# plt.axis("off")
# plt.tight_layout(pad = 10)
 
# plt.show()
